In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv
import random

def init_driver():
    # Initialize the WebDriver
    driver = webdriver.Chrome()
    return driver

from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def scrape_page(driver, page_number):
    url = f"https://zkevm.polygonscan.com/batches?p={page_number}"
    driver.get(url)
    time.sleep(random.randint(0, 2))  # Wait for the page to load

    # Locating the table rows
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')

    data = []
    for row in rows:
        # Extract each column's data
        # Assuming the columns are in the order: 'Index', 'L1 Block', 'L1 Txn Hash', 'Txn'
        columns = row.find_elements(By.TAG_NAME, 'td')

        if len(columns) >= 4:  # Ensure there are enough columns
            batch_index = columns[0].text
            txn = columns[4].text
            l1_hash = columns[7].text

            # Append the extracted data to the data list
            data.append({
                'batch_index': batch_index,
                'Txn': txn,
                'l1_block':l1_hash
            })

    return data

def save_to_csv(data, filename):
    # Assuming data is a list of dictionaries
    if data and isinstance(data[0], dict):
        keys = data[0].keys()
        with open(filename, 'w', newline='', encoding='utf-8') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data)

def main():
    driver = init_driver()
    results = []

    try:
        for page in range(50400, 51187):
            data = scrape_page(driver, page)
            results.extend(data)

            if (page - 1500) % 300 == 0:  # Save every 100 iterations
                # Construct a unique filename
                filename = f'scraped_data_{page}.csv'
                save_to_csv(results, filename)
                results = []  # Reset the results list after saving

            time.sleep(0.5)  # Delay to avoid rate limiting

        # Save the remaining data, if any
        if results:
            save_to_csv(results, 'scraped_data_final.csv')

    finally:
        driver.quit()

if __name__ == "__main__":
    main()